In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("./Dataset/E-commerce(split_session).csv")
print(df.columns)
df.head()

AttributeError: module 'pandas' has no attribute 'read_csv'

### purchase_count 이상치 제거

In [14]:
new_row = df.loc[df["purchase_count"] >20]
df.drop(new_row.index, axis=0, inplace=True)

NameError: name 'df' is not defined

In [4]:
df["purchase_count"].value_counts()

NameError: name 'df' is not defined

### price 이상치 제거

In [5]:
user_freq = df["price"]
Q1 = user_freq.quantile(.25)
Q3 = user_freq.quantile(.75)
IQR = Q3 - Q1
Q2 = user_freq.quantile(.5)
Q4 = user_freq.quantile(1)

outlier = Q3 + 1.5 * IQR
print(outlier)
print(len(df.loc[df["price"]>outlier])/len(df)*100)

1039.9099999999999
8.137838804484737


In [6]:
new_row = df.loc[df["price"] > outlier]
df.drop(new_row.index, axis=0, inplace=True)

### price 정규화

In [7]:
mean = df["price"].mean()
std = df["price"].std()
df["price"] = df["price"].apply(lambda x: (x-mean)/ std)

In [8]:
print(mean)
print(std)

285.5249790125641
252.9166234912892


In [9]:
df.head()

,price,category_accessories,category_apparel,category_appliances,category_auto,category_computers,category_construction,category_country_yard,category_electronics,category_furniture,...,dayofweek_Mon,dayofweek_Tus,dayofweek_Wed,dayofweek_Thu,dayofweek_Fri,dayofweek_Sat,dayofweek_Sun,purchase_count,session,total_session
0,-0.928428,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,-0.399361,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,-0.163077,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,-0.905140,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,3
4,-0.554946,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,3


In [10]:
len(df)

4430535

# customer_rayalty

### total_session 빈도에 따라 customer_royalty로 변경

In [12]:
new_df = df.copy()

In [13]:
df['total_session'].value_counts(normalize=True)

0      0.226879
1      0.154380
2      0.105154
3      0.074842
4      0.055616
         ...   
192    0.000028
199    0.000027
132    0.000019
240    0.000012
299    0.000009
Name: total_session, Length: 247, dtype: float64

In [14]:
#customer_royalty 0~9까지 10개의 구간으로 나눈다. 0 ~ 4 까지는 total_session에 따라 분리, 이후부터 구간을 설정하여 구분

In [15]:
new_df["customer_royalty"] = new_df["total_session"].apply(lambda x: 'Bronze' if (x == 0 ) else x)

In [16]:
new_df["customer_royalty"] = new_df["customer_royalty"].apply(lambda x: 'Silver' if (x >= 1 and x <= 3 ) else x)

In [17]:
new_df["customer_royalty"] = new_df["customer_royalty"].apply(lambda x: 'Gold' if (x >= 4 and x <= 10 ) else x)

In [18]:
new_df["customer_royalty"] = new_df["customer_royalty"].apply(lambda x: 'Platinum' if (x >= 11 and x <= 39) else x)

In [19]:
new_df["customer_royalty"] = new_df["customer_royalty"].apply(lambda x: 'Dia' if (x >= 40) else x)

In [20]:
new_df["customer_royalty"].value_counts(normalize=True, sort=False)

0    0.226879
1    0.334376
2    0.220781
3    0.157475
4    0.060489
Name: customer_royalty, dtype: float64

In [21]:
#total_session대신 customer_royalty를 사용
new_df.drop(["total_session"], axis=1, inplace=True)

In [22]:
len(new_df)

4430535

In [23]:
new_df.to_csv("./Dataset/E-commerce(royalty).csv", index=False)